# "Fallecidos confirmados por comuna"
> Fallecidos confirmados por comuna.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import display, Markdown, display_html, HTML

In [2]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)

In [3]:
#hide_input
display(Markdown(f"Última actualización: {update_date.strftime('%d/%m/%Y')}."))

Última actualización: 25/06/2020.

# Evolución del total de fallecidos confirmados por comuna

In [4]:
#collapse-hide
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import display, Markdown, display_html, HTML

#download the data
county_deaths_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna.csv")
#select Metropolitan region
county_deaths_RM_raw = county_deaths_raw.query(
    "Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Codigo comuna"]).set_index("Comuna")
#first and last report date
first_date = county_deaths_RM_raw.T.iloc[1].name
last_date = county_deaths_raw.T.iloc[-1].name
#build dataframe with all the dates since first date to last date
county_deaths_RM = pd.DataFrame()
county_deaths_RM["Fecha"] = pd.date_range(start=first_date, end=last_date).strftime('%Y-%m-%d')
county_deaths_RM = county_deaths_RM.set_index("Fecha")
#transpose the dataframe
county_deaths_RM_raw = county_deaths_RM_raw.T
#build the dataframe with all the data from the reports
county_deaths_RM[list(county_deaths_RM_raw.columns)] = county_deaths_RM_raw[list(county_deaths_RM_raw.columns)]

#data in long form to plot it on Altair
county_deaths_RM_long_form = county_deaths_RM.drop(
    columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Fallecidos confirmados")

#plot the data
input_dropdown = alt.binding_select(options=county_deaths_RM_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(county_deaths_RM_long_form.fillna(0)).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de fallecidos confirmados")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title="Total fallecidos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

# Evolución del total de fallecidos confirmados por comuna (interpolación)

In [5]:
#collapse-hide
#linear interpolation
county_deaths_RM = np.round(county_deaths_RM.interpolate(method="linear"))

#data in long form to plot it on altair
county_deaths_RM_long_form = county_deaths_RM.drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Fallecidos confirmados")
#plot the data
input_dropdown = alt.binding_select(options=county_deaths_RM_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(county_deaths_RM_long_form.fillna(0)).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de fallecidos confirmados")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title="Total fallecidos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

# Evolución de nuevos fallecidos confirmados por comuna (interpolación)

In [6]:
#collapse-hide
new_county_deaths_RM = county_deaths_RM.diff().drop("2020-06-12")

#data in long form to plot it on altair
new_county_deaths_RM_long_form = new_county_deaths_RM.drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Fallecidos confirmados")
#plot the data
input_dropdown = alt.binding_select(options=new_county_deaths_RM_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(new_county_deaths_RM_long_form.fillna(0)).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de fallecidos confirmados")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title="Nuevos fallecidos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

In [7]:
#hide
new_county_deaths_RM.sum(axis=1)

Fecha
2020-06-13    277.0
2020-06-14    268.0
2020-06-15    277.0
2020-06-16    335.0
2020-06-17    342.0
2020-06-18    336.0
2020-06-19    335.0
2020-06-20    174.0
2020-06-21    182.0
2020-06-22    174.0
2020-06-23    174.0
dtype: float64